In [ ]:
from chromatic import *
import emcee
import corner
import speclite as speclite
import glob as glob
from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from scipy.optimize import minimize
from scipy.optimize import curve_fit

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (6, 4),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
plt.rcParams.update(params)
plt.style.use('tableau-colorblind10')

In [ ]:
ordernumbers = np.arange(53,84)
bad_orders = np.array([56,57,59,60,62,63,64,65,66,67,68,69,
                       72,73,74,75,78,79,80,83])
good_orders = np.array([53,54,55,58,61,70,71,76,77,81,82])

# nsteps = 1000
# modeltype = 'Final_Teff_Phot'
# modelname = 'Teff + Phot'
# visits = ['F21','S22']
# orders_I_care_about = [52]

# nsteps = 1000
# modeltype = 'Final_Teff_Spec'
# modelname = 'Teff + Spec'
# visits = ['F21','S22','combined']
# orders_I_care_about = good_orders

# nsteps = 2000
# modeltype = f'Final_Teff_Phot_Spec'
# modelname = 'Teff + Phot + Spec'
# suptitle = r'T$_{\rm{eff}}$ & Photometric & S($\lambda$) Model Results'
# visits = ['F21','S22']
# orders_I_care_about = good_orders

nsteps = 3000
modeltype = f'Final_Teff_Phot_Spec'
modelname = 'Teff + Phot + Spec'
suptitle = r'T$_{\rm{eff}}$ & Photometric & S($\lambda$) Model Results'
visits = ['combined']
orders_I_care_about = good_orders

for visit in tqdm(visits):
                
    for order in orders_I_care_about:
        label = f'{visit}_{order}_{modeltype}'
        samples_file_label = label+f'_{nsteps}steps'
        '''
        READ IN THE MCMC SAMPLES
        '''
        reader = emcee.backends.HDFBackend(f'../data/samples/{samples_file_label}.h5')
        sampler = reader.get_chain(discard=int(0.25*nsteps), flat=True)
        samples = sampler.reshape((-1, 4)).T
        fspot_sam, dfspot_sam, Tspot_sam, Tamb_sam = samples
        sig1_fspot = np.percentile(fspot_sam, [15.9, 50., 84.1]) # central 1-sigma values
        sig1_dfspot = np.percentile(dfspot_sam, [15.9, 50., 84.1])
        sig1_Tspot = np.percentile(Tspot_sam, [15.9, 50., 84.1])
        sig1_Tamb = np.percentile(Tamb_sam, [15.9, 50., 84.1])
        # Define the 50th percentile and 1-sigma interval parameter values from the samples
        best_params = np.array([sig1_fspot[1], sig1_dfspot[1], sig1_Tspot[1], sig1_Tamb[1]])
        best_params_err_lower = best_params-[sig1_fspot[0], sig1_dfspot[0], sig1_Tspot[0], sig1_Tamb[0]]
        best_params_err_higher = [sig1_fspot[2], sig1_dfspot[2], sig1_Tspot[2], sig1_Tamb[2]]-best_params
        variable_names = [r'f$_{\rm{spot}}$',r'$\Delta$f$_{\rm{spot}}$',r'T$_{\rm{spot}}$',r'T$_{\rm{amb}}$']
        normie_names = ['fspot','dfspot','Tspot','Tamb']        

        """
        Now we will plot samples from the posteriors
        """
        wavelength=np.linspace(0.5,5,200)*u.micron
        plt.figure(figsize=(6,4))
        title_label=f'{visit} Order {order} {modelname}'
        plt.title(title_label,fontsize=16,loc='left')        
        plt.ylabel(r'$\rm{\Delta~D(\lambda)_{spot}}/(\frac{R_p}{R_*})^2+1$',fontsize=15)
        plt.xlabel(r'Wavelength ($\mu$m)',fontsize=20)
        plt.ylim(0.75,1.8)
        plt.xlim(0.5,5)
        plt.axhline(1.02,color='teal',label='1 Scale Height',linestyle='--')
        plt.axhline(1.0,color='k',label='Opaque transit depth')
        for k in range(0,1000):
            j = np.random.randint(low=0,high=(len(Tspot_sam)-1))
            transit_depth = (0.0512)**2.
            s_spot = get_phoenix_photons(temperature=Tspot_sam[j], wavelength = wavelength,
                                                           logg=4.4, metallicity=0.0)
            s_amb = get_phoenix_photons(temperature=Tamb_sam[j], wavelength = wavelength,
                                                           logg=4.4, metallicity=0.0)
            flux_ratio = s_spot[1]/s_amb[1]
            top = 1.
            bottom = (1. - fspot_sam[j]) + fspot_sam[j] * flux_ratio
            delta_D_spot = ((top / bottom) - 1.) * transit_depth
            depth_factor = (delta_D_spot/transit_depth) + 1.
            plt.plot(s_spot[0],depth_factor,color = 'firebrick',alpha=0.02) # this will be the input wavelength from the order in question
        plt.legend(loc = 'upper right')
        plt.savefig(f'../figs/{label}_contam.png',dpi=200)
        plt.show()
        plt.close()